In [ ]:
'''def PercentageGrowth(n,m, addonfactor, baseline_mtm, baseline_notional, MtM_min, MtM_max):   

    # Randomly generate our dataset
    # update_position_impacts(n, baseline_mtm, baseline_notional, MtM_min, MtM_max)
    # positions = position_impacts_state["data"]
    
    positions = get_position_impacts(n, baseline_mtm, baseline_notional, MtM_min, MtM_max)

    singleposition = False

    # Set up the following lists to store the exposures for each trade
    bf_list = []
    cons_list = []
    lin_list = []
    avg_list = []

    # Add the initial exposure for the baseline trade
    bf_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))
    lin_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))
    cons_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))
    avg_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))


    # We iterate through the list. Once we hit m trades, we update the baseline and repeat
    for i in range(int(n/m)):
        
        cumulative_positions = np.zeros((0, 2))  # empty to start
        cumulative_positions = np.vstack([cumulative_positions, positions[0, :]])

        for j in range(m):
            
            # Add the next position to the cumulative set
            cumulative_positions = np.vstack([cumulative_positions, positions[j+1 + m*i, :]])

            # Perform the PSR calculations for each strategy
            NGR_BF = PSR_NGR(cumulative_positions, addonfactor, singleposition)
            NGR_Lin = PSR_Linear(cumulative_positions, addonfactor)
            NGR_Cons = PSR_Conservative(cumulative_positions, addonfactor)
            NGR_Avg = PSR_Average(cumulative_positions, addonfactor, j+1)

            # Append results to respective lists
            bf_list.append(NGR_BF)
            lin_list.append(NGR_Lin)
            cons_list.append(NGR_Cons)
            avg_list.append(NGR_Avg)

        # Re-calculate the baseline by adding in the processed impacts
        mtmRefresh = np.sum(cumulative_positions[:,0])
        notionalRefresh = np.sum(cumulative_positions[:,1])

        # Update the first row (baseline)
        positions[0,0] = max(0,mtmRefresh)
        positions[0,1] = notionalRefresh

        # Add this baseline calculation to the dataset
        bf_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))
        lin_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))
        cons_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))
        avg_list.append(PSR_NGR(np.array([positions[0,:]]), addonfactor, singleposition))

    #print(remove_decimal_places(bf_list))

    # POST-PROCESSING
    # Below we plot our results
    x = np.linspace(0, n+int(n/m), n+int(n/m)+1)




    if n % m == 0:
        
        lin_diff = [(lin_list[i] - bf_list[i])/bf_list[i]*100 for i in range(len(bf_list))]
        cons_diff = [(cons_list[i] - bf_list[i])/bf_list[i]*100 for i in range(len(cons_list))]
        avg_diff = [(avg_list[i] - bf_list[i])/bf_list[i]*100 for i in range(len(avg_list))]


        plt.figure(figsize=(8,5))
        plt.grid(True)
        plt.plot(x, cons_diff, 'r', label = "Cons. Diff.")
        plt.plot(x, lin_diff, 'g', label = "Lin. Diff")
        plt.plot(x, avg_diff, 'y', label = "Avg. Diff")
        plt.xlabel("Trade Number")
        plt.ylabel("Percantage Difference")

        # We will add in vertical lines to indicate the points in the dataset when a refresh is carried out
        for i in range(int(n/m)):

            vline_x = (i+1)*(m+1)
            ymin, ymax = plt.ylim()
            plt.axvline(vline_x, color='k', linestyle='--')
            vline_x2 = (i+1)*(m+1)-1
            ymin2, ymax2 = plt.ylim()
            plt.axvline(vline_x2, color='r', linestyle='--')
            plt.text(vline_x, ymax + (ymax - ymin)*0.025, 'RF', ha="center", va = "bottom", color="k")
            plt.text(vline_x2, ymax2 + (ymax2 - ymin2)*0.025, '↻', ha="center", va = "bottom", color="r")

        vline_start = 0
        ymin3, ymax3 = plt.ylim()
        plt.axvline(vline_start, color='b', linestyle='--')
        plt.text(vline_start, ymax3 + (ymax3 - ymin3)*0.025, 'Base', ha="center", va = "bottom", color="b")

        plt.title("Percentage Difference From BF", pad=40)
        plt.legend()
        plt.subplots_adjust(hspace=0.5)
        plt.show()

    else:
        print("\n")
        print("ERROR: you must choose n and m such that m divides n (ie. n / m is a whole number)")


'''

'''
interact(PercentageGrowth,
         n=widgets.IntSlider(min=1, max=100, step=1, value=10, description="# of Trades"),
         m=widgets.IntSlider(min=1, max=100, step=1, value =5, description="Refresh Cycle"),
         addonfactor = widgets.FloatSlider(min=0,max=1,step=0.01,value=0.01,description="AddOnFactor"),
         baseline_mtm = widgets.IntSlider(min=10, max=1000, step=5, value=100, description="MTM Baseline"),
         baseline_notional = widgets.IntSlider(min=1000, max=100000, step=100, value=10000, description="Notional Baseline"),
         MtM_min = widgets.IntSlider(min=-1000, max=-50, step=1, value =-100, description="MtM Min"),         
         MtM_max = widgets.IntSlider(min=50, max=1000, step=1, value=100, description="MtM Max"),
         )
'''

'''# Define your sliders
n_slider = widgets.IntSlider(min=1, max=100, step=1, value=10, description="# of Trades")
m_slider = widgets.IntSlider(min=1, max=100, step=1, value=5, description="Refresh Cycle")
addonfactor_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.01, description="AddOnFactor")
baseline_mtm_slider = widgets.IntSlider(min=100, max=10000, step=20, value=1000, description="MTM Baseline")
baseline_notional_slider = widgets.IntSlider(min=10000, max=1000000, step=100, value=100000, description="Notional Baseline")
MtM_min_slider = widgets.IntSlider(min=-1000, max=-10, step=10, value=-50, description="MtM Min")
MtM_max_slider = widgets.IntSlider(min=10, max=1000, step=1, value=50, description="MtM Max")

# Function to update slider values based on button press
def set_preset_1(b):
    n_slider.value = 30
    m_slider.value = 10
    addonfactor_slider.value = 0.01
    baseline_mtm_slider.value = 10000
    baseline_notional_slider.value = 500000
    MtM_min_slider.value = -20
    MtM_max_slider.value = 20

def set_preset_2(b):
    n_slider.value = 30
    m_slider.value = 10
    addonfactor_slider.value = 0.1
    baseline_mtm_slider.value = 1000
    baseline_notional_slider.value = 100000
    MtM_min_slider.value = -100
    MtM_max_slider.value = 100

# Create buttons for presets
preset_button_1 = widgets.Button(description="Large Baseline, Small Impacts")
preset_button_2 = widgets.Button(description="Small Baseline, Large Impacts")

# Assign button click event to set preset values
preset_button_1.on_click(set_preset_1)
preset_button_2.on_click(set_preset_2)

# Make the buttons larger
preset_button_1.layout = Layout(width='250px', height='50px')
preset_button_2.layout = Layout(width='250px', height='50px')

# Create a grid layout for the sliders (2x3 layout)
grid_layout = widgets.GridBox(
    children=[n_slider, m_slider, addonfactor_slider, baseline_mtm_slider, baseline_notional_slider, MtM_min_slider, MtM_max_slider],
    layout=Layout(grid_template_columns="repeat(3, 300px)", grid_template_rows="repeat(3, auto)", grid_gap="10px")
)

# Layout for buttons
button_layout = widgets.HBox([preset_button_1, preset_button_2])

# Display the interactive widgets and buttons in a grid layout
ui = widgets.VBox([grid_layout, button_layout])
out = widgets.interactive_output(PercentageGrowth, {
    'n': n_slider, 
    'm': m_slider, 
    'addonfactor': addonfactor_slider, 
    'baseline_mtm': baseline_mtm_slider, 
    'baseline_notional': baseline_notional_slider, 
    'MtM_min': MtM_min_slider, 
    'MtM_max': MtM_max_slider
})

display(ui, out)
'''